# Исследование объявлений о продаже квартир

## Общая информация

**Входные данные**: архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет (данные сервиса Яндекс Недвижимость). 

**Два вида данных**:
1. вписаны пользователем; 
2. получены автоматически на основе картографических данных.

Нужно **научиться** определять рыночную стоимость объектов недвижимости. 

**Задача**: установить параметры для построения автоматизированной системы: она отследит аномалии и мошенническую деятельность.

## Чтение данных

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

from datetime import datetime as dt

<div class="alert alert-block alert-info">
<b>Tip:</b> Если вы запускаете Jupyter Notebook на локальном ПК, можно использовать блок `try...except`, чтобы при проверке вашего проекта в среде Яндекса, у ревьюера не возникала ошибка, если вы вдруг забыли изменить путь к файлу :) </div>

In [2]:
try:
    df = pd.read_csv('/datasets/data.csv')
except:
    df = pd.read_csv('real_estate_data.csv', sep='\t', parse_dates=['first_day_exposition'])

### Изучаем на данные

#### Описание полей данных

- *airports_nearest* — расстояние до ближайшего аэропорта в метрах (м)
- *balcony* — число балконов
- *ceiling_height* — высота потолков (м)
- *cityCenters_nearest* — расстояние до центра города (м)
- *days_exposition* — сколько дней было размещено объявление (от публикации до снятия)
- *first_day_exposition* — дата публикации
- *floor* — этаж
- *floors_total* — всего этажей в доме
- *is_apartment* — апартаменты (булев тип)
- *kitchen_area* — площадь кухни в квадратных метрах (м²)
- *last_price* — цена на момент снятия с публикации
- *living_area* — жилая площадь в квадратных метрах (м²)
- *locality_name* — название населённого пункта
- *open_plan* — свободная планировка (булев тип)
- *parks_around3000* — число парков в радиусе 3 км
- *parks_nearest* — расстояние до ближайшего парка (м)
- *ponds_around3000* — число водоёмов в радиусе 3 км
- *ponds_nearest* — расстояние до ближайшего водоёма (м)
- *rooms* — число комнат
- *studio* — квартира-студия (булев тип)
- *total_area* — площадь квартиры в квадратных метрах (м²)
- *total_images* — число фотографий квартиры в объявлении

In [3]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
pd.options.display.max_columns = 50

In [5]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропуски в полях:
- *ceiling_height* — высота потолков (м)
- *floors_total* — всего этажей в доме
- *living_area* — жилая площадь в квадратных метрах (м²)
- *is_apartment* — апартаменты (булев тип)
- *kitchen_area* — площадь кухни в квадратных метрах (м²)
- *balcony* — число балконов
- *locality_name* — название населённого пункта
- *airports_nearest* — расстояние до ближайшего аэропорта в метрах (м)
- *cityCenters_nearest* — расстояние до центра города (м)
- *parks_around3000* - число парков в радиусе 3 км
- *parks_nearest* — расстояние до ближайшего парка (м)
- *ponds_around3000* — число водоёмов в радиусе 3 км
- *ponds_nearest* — расстояние до ближайшего водоёма (м)
- *days_exposition* — сколько дней было размещено объявление (от публикации до снятия)

## Предобработка данных

### Заполнение пропусков

**1. Определите и изучите пропущенные значения**

- Для некоторых пропущенных значений можно предположить логичную замену. ***Например, если человек не указал число балконов — скорее всего, их нет. Такие пропуски правильно заменить на 0.*** Для других типов данных нет подходящего значения на замену. В этом случае правильно оставить эти значения пустыми. Отсутствие значения — тоже важный сигнал, который не нужно прятать.
- Заполните пропуски, где это уместно. Опишите, почему вы решили заполнить пропуски именно в этих столбцах и как выбрали значения.
- Укажите причины, которые могли привести к пропускам в данных.

**2. Приведите данные к нужным типам**
- Поясните, в каких столбцах нужно изменить тип данных и почему.

**Вопросы**:
- Изучите следующие параметры: **площадь (total_area), цена (last_price), число комнат (rooms), высота потолков (ceiling_height)**. 
- Изучите **время продажи квартиры**.
- Изучите, зависит ли цена от **площади, числа комнат, удалённости от центра (cityCenters_nearest)**. 
- Изучите зависимость цены от того, **на каком этаже (floor) расположена квартира: первом, последнем или другом**.
- Изучите зависимость от **даты размещения (first_day_exposition): дня недели, месяца и года**.
- Выберите 10 **населённых пунктов (locality_name)** с наибольшим числом объявлений.

In [7]:
def nan_values(df):
    count_missing = df.isna().sum()
    percent_missing = round(df.isna().sum() * 100 / len(df), 2)
    
    missing_value_df = pd.DataFrame({'num_missing': count_missing,
                                     'percent_missing': percent_missing})
    
    return missing_value_df

nan_values(df)

,num_missing,percent_missing
children,0,0.0
days_employed,2174,10.1
dob_years,0,0.0
education,0,0.0
education_id,0,0.0
family_status,0,0.0
family_status_id,0,0.0
gender,0,0.0
income_type,0,0.0
debt,0,0.0


#### Балконы (balcony)

In [8]:
df['balcony'] = df['balcony'].fillna(0).astype(int)

KeyError: 'balcony'

#### Апартаменты (is_apartment)

In [ ]:
df['is_apartment'] = df['is_apartment'].fillna(False)

#### Кол-во дней размещения объявления (days_exposition)

- Пропуски — еще не снятые с публикации объявления?
- Подумать, чем заменить :) Лучше, чем today(). Возможно, вычесть из максимальной имеющейся даты. Или не заменять совсем, чтобы не исказить данные (объяснить это в комментарии для ревьюера).

In [ ]:
df['days_exposition'].fillna((dt.today() - df['first_day_exposition']).dt.days, inplace=True)

#### Высота потолков (ceiling_height)

In [ ]:
df['ceiling_height'].describe().T

In [ ]:
df.groupby('locality_name').agg(
    min_height=('ceiling_height', 'min'),
    max_height=('ceiling_height', 'max'),
    mean_height=('ceiling_height', 'mean'),
    median_height=('ceiling_height', 'median')
)

In [ ]:
df['ceiling_height'].fillna(df.groupby('locality_name')['ceiling_height'].transform('median'), inplace=True)

#### Площадь кухни (kitchen_area) и жилая площадь (living_area)

In [ ]:
# kitchen_total = (df['kitchen_area'] / df['total_area']).mean()
# living_total = (df['living_area'] / df['total_area']).mean()

kitchen = (df['kitchen_area'] / df['total_area']).mean() * df['total_area']
living = (df['living_area'] / df['total_area']).mean() * df['total_area']

df.fillna({'kitchen_area': kitchen, 'living_area': living}, inplace=True)

___

In [ ]:
# df.dropna(subset=['column_name'], inplace=True)

## Посчитайте и добавьте в таблицу

- цену квадратного метра;
- день недели, месяц и год публикации объявления;
- этаж квартиры; варианты — первый, последний, другой;
- соотношение жилой и общей площади, а также отношение площади кухни к общей.

In [ ]:
# Цена за кв. м.

df['per_square_meter'] = df['last_price'] / df['total_area']

In [ ]:
# День недели, месяц и год публикации объявления

df['day_of_week'] = df['first_day_exposition'].dt.dayofweek
df['month'] = df['first_day_exposition'].dt.month
df['year'] = df['first_day_exposition'].dt.year

In [ ]:
# Этаж квартиры; варианты — первый, последний, другой

col = 'floor'
conditions = [df[col] == 1,
              df[col] >= df['floors_total']]

choices = ['первый', 'последний']
    
df['floor_category'] = np.select(conditions, choices, default='другой') # default = else

In [ ]:
# Соотношение жилой и общей площади
df['living_total'] = df['living_area'] / df['total_area']

# Соотношение площади кухни и общей площади
df['kitchen_total'] = df['kitchen_area'] / df['total_area']

## Проведите исследовательский анализ данных и выполните инструкции

- Изучите следующие параметры: площадь, цена, число комнат, высота потолков. Постройте гистограммы для каждого параметра.
- Изучите время продажи квартиры. Постройте гистограмму. Посчитайте среднее и медиану. Опишите, сколько обычно занимает продажа. Когда можно считать, что продажи прошли очень быстро, а когда необычно долго?
- Уберите редкие и выбивающиеся значения. Опишите, какие особенности обнаружили.
- Какие факторы больше всего влияют на стоимость квартиры? Изучите, зависит ли цена от площади, числа комнат, удалённости от центра. Изучите зависимость цены от того, на каком этаже расположена квартира: первом, последнем или другом. Также изучите зависимость от даты размещения: дня недели, месяца и года.
- Выберите 10 населённых пунктов с наибольшим числом объявлений. Посчитайте среднюю цену квадратного метра в этих населённых пунктах. Выделите населённые пункты с самой высокой и низкой стоимостью жилья. Эти данные можно найти по имени в столбце locality_name.
- Изучите предложения квартир: для каждой квартиры есть информация о расстоянии до центра. Выделите квартиры в Санкт-Петербурге (locality_name). Ваша задача — выяснить, какая область входит в центр. Создайте столбец с расстоянием до центра в километрах: округлите до целых значений. После этого посчитайте среднюю цену для каждого километра. Постройте график: он должен показывать, как цена зависит от удалённости от центра. Определите границу, где график сильно меняется, — это и будет центральная зона.
- Выделите сегмент квартир в центре. Проанализируйте эту территорию и изучите следующие параметры: площадь, цена, число комнат, высота потолков. Также выделите факторы, которые влияют на стоимость квартиры (число комнат, этаж, удалённость от центра, дата размещения объявления). Сделайте выводы. Отличаются ли они от общих выводов по всей базе?

#### Изучение параметров

Изучите следующие параметры: площадь, цена, число комнат, высота потолков. Постройте гистограммы для каждого параметра.

In [ ]:
def histplt(df, columns):
    fig, axes = plt.subplots(2, 2, figsize=(16,10))

    axe = axes.ravel()

    for i, column in enumerate(columns):
        df[column].hist(ax=axe[i], bins=50, color='#32B5C9').set_title(column)

In [ ]:
histplt(df, ['total_area', 'last_price', 'rooms', 'ceiling_height'])

In [ ]:
df[['total_area', 'last_price', 'rooms', 'ceiling_height']].describe().T

In [ ]:
def pxplt(df, columns):
    for column in columns:
        fig = px.histogram(df, x=column, title=column, nbins=500)
        fig.show()

In [ ]:
pxplt(df, ['total_area', 'last_price', 'rooms', 'ceiling_height'])

##### Выводы

- **Площадь квартир**

Наибольшее число объявлений — квартиры 30-70 кв. м., сильно меньшее кол-во квартир площадью более 200 кв. м.

- **Цена продажи**

Наибольшее число квартир имеют стоимость от 3 до 7 млн. руб., редкость — квартиры стоимостью более 30 млн. руб.

- **Кол-во комнат**

Преобладание 1- и 2-комнатных квартир, 3-комнатные также популярны.

- **Высота потолков**

Высота потолков в большинстве квартир — от 2.5 до 2.8 м. Квартиры с потолками более 3.5 метров встречаются намного реже.

По аналогии можно посмотреть остальные параметры и сделать выводы:
- цена за кв. м.
- наличие балкона
- этаж

In [ ]:
df[['floor', 'balcony']].describe().T

#### Изучение времени продажи квартиры

- Изучите время продажи квартиры. Постройте гистограмму. Посчитайте среднее и медиану. Опишите, сколько обычно занимает продажа. Когда можно считать, что продажи прошли очень быстро, а когда необычно долго?

In [ ]:
column = 'days_exposition'

df[column].hist(bins=50, color='#32B5C9', figsize=(8,6)).set_title(column);

In [ ]:
fig = px.histogram(df, x=column, title='Время продажи квартиры', nbins=100)
fig.show()

In [ ]:
fig = px.box(df, y=column)
fig.show()

In [ ]:
df[['days_exposition']].describe().T

##### Выводы

- Большинство квартир продаются в течение 3-х месяцев, однако 25% квартир остаются опубликованными около 7-8 месяцев. 
- Быстро: **<= 45 дней**? Долго **>= 232 дня**?

#### Редкие и выбивающиеся значения

In [ ]:
df[['total_area', 'rooms', 'last_price', 'ceiling_height']].describe().loc[['min', 'max']].T

##### Высота потолков

- Что делать с высотой потолков 1 м / 100 м?

In [ ]:
df[df['ceiling_height'] > 2] # > 10 м

In [ ]:
df = df[(df['ceiling_height'] > 2) & (df['ceiling_height'] < 10)]

##### Цена продажи

In [ ]:
fig = px.box(df, y='last_price')
fig.show()

In [ ]:
df[['last_price']].describe().T

In [ ]:
df = df[(df['last_price'] >= 1000000)] # & (df['last_price'] < 160000000)]

##### Общая площадь

In [ ]:
df[['total_area']].describe().T

#### Факторы, влияющие на стоимость квартиры

- Какие факторы больше всего влияют на стоимость квартиры? Изучите, зависит ли цена от площади, числа комнат, удалённости от центра. Изучите зависимость цены от того, на каком этаже расположена квартира: первом, последнем или другом. Также изучите зависимость от даты размещения: дня недели, месяца и года.

##### Зависимость стоимости от площади

In [ ]:
fig = px.scatter(df, x='total_area', y='last_price', title='Зависимость стоимости от площади')
fig.show()

##### Зависимость стоимости от количества комнат

In [ ]:
rooms_price = df.pivot_table(
    index='rooms',
    values='last_price',
    aggfunc='median'
).reset_index()

fig = px.line(rooms_price, x='rooms', y='last_price', title='Зависимость стоимости от количества комнат')
fig.show()

##### Зависимость стоимости от этажа

In [ ]:
floor_price = df.pivot_table(
    index='floor_category',
    values='last_price',
    aggfunc='median'
).reset_index()

fig = px.line(floor_price, x='floor_category', y='last_price', title='Зависимость стоимости от этажа')
fig.show()

##### Выводы

- Стоимость квартир в зависимости от площади.
- Стоимость квартиры зависит от кол-ва комнат.
- Наиболее низкой стоимостью обладают квартиры, расположенные на первом этаже.

При исследовании зависимости от временного периода полезно подкрепить выводы данными из реальной жизни, которые могли повлиять на стоимость квартир в тот или иной период (если зависимость есть). Например, зависимость от года: экономическая ситуация меняется в зависимости от тех или иных событий, а вместе с ней меняется покупательская способность.

#### 10 населённых пунктов с наибольшим числом объявлений
- Выберите 10 населённых пунктов с наибольшим числом объявлений. Посчитайте среднюю цену квадратного метра в этих населённых пунктах. Выделите населённые пункты с самой высокой и низкой стоимостью жилья. Эти данные можно найти по имени в столбце locality_name.

In [ ]:
df.groupby('locality_name')['per_square_meter'].agg(['count', 'mean']).sort_values(by='count')[::-1].head(10).sort_values(by='mean')[::-1]

##### Выводы
- Наибольшая ср. цена за кв. м.
- Наименьшая ср. цена за кв. м.
- Объяснение

#### Центр
- Изучите предложения квартир: для каждой квартиры есть информация о расстоянии до центра. Выделите квартиры в Санкт-Петербурге (locality_name). Ваша задача — выяснить, какая область входит в центр. Создайте столбец с расстоянием до центра в километрах: округлите до целых значений. После этого посчитайте среднюю цену для каждого километра. Постройте график: он должен показывать, как цена зависит от удалённости от центра. Определите границу, где график сильно меняется, — это и будет центральная зона.
- Выделите сегмент квартир в центре. Проанализируйте эту территорию и изучите следующие параметры: площадь, цена, число комнат, высота потолков. Также выделите факторы, которые влияют на стоимость квартиры (число комнат, этаж, удалённость от центра, дата размещения объявления). Сделайте выводы. Отличаются ли они от общих выводов по всей базе?

##### Определение центральной зоны

In [9]:
df['cityCenters_nearest_km'] = (df['cityCenters_nearest'] / 1000).round()

spb = df.query('locality_name == "Санкт-Петербург"')

cityCenter_price = spb.pivot_table(
    index='cityCenters_nearest_km', values='last_price', aggfunc='mean'
).reset_index()

fig = px.line(
    cityCenter_price, 
    x='cityCenters_nearest_km', 
    y='last_price', 
    title='Зависимость стоимости от приближенности к центру')
fig.show()

KeyError: 'cityCenters_nearest'

###### Выводы
- Центр - 3 км?

##### Сегмент квартир в центре

- Выделите сегмент квартир в центре. Проанализируйте эту территорию и изучите следующие параметры: площадь, цена, число комнат, высота потолков. Также выделите факторы, которые влияют на стоимость квартиры (число комнат, этаж, удалённость от центра, дата размещения объявления). Сделайте выводы. Отличаются ли они от общих выводов по всей базе?

In [ ]:
histplt(spb, ['total_area', 'last_price', 'rooms', 'ceiling_height'])

In [ ]:
spb[['total_area', 'last_price', 'rooms', 'ceiling_height']].describe().T

In [ ]:
center = spb.query('cityCenters_nearest_km <= 3')

histplt(center, ['total_area', 'last_price', 'rooms', 'ceiling_height'])

In [ ]:
center[['total_area', 'last_price', 'rooms', 'ceiling_height']].describe().T

In [ ]:
pxplt(center, ['total_area', 'last_price', 'rooms', 'ceiling_height'])

###### Выводы
- Квартиры в центре просторнее.
- В центре больше 2- и 3-комнатных квартир, тогда как в целом в Санкт-Петербурге много 1-комнатных квартир.
- Медианная высота потолков в центре больше, чем в остальных частях города.
- Объяснение

## Ответы на вопросы

In [ ]:
x = 4
x==4

In [ ]:
x==4 and x%2==0 # True and True

In [ ]:
if x==4 and x%2==0:
    print(1)

In [ ]:
df['rooms'] > 2

In [ ]:
df[(df['rooms'] > 2) & (df['floor'] > 2)]

In [ ]:
(df['rooms'] > 2).any()

In [ ]:
(df['rooms'] > 2).any() and (df['floor'] > 2).any()

___

In [ ]:
df = pd.DataFrame({'value1': np.random.normal(1, 1, 99),
                   'value2': [-1]*33 + [0]*33 + [1]*33})

df.hist();

In [ ]:
df.plot.hist();

In [ ]:
pd.plotting.scatter_matrix(df, alpha=0.2);

In [ ]:
df.plot.scatter(x='value1', y='value2');